In [1]:
# Importações necessárias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import imageio
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
import warnings
import os # Importar para lidar com caminhos de pastas

warnings.filterwarnings('ignore')

In [2]:
# Gerar dados sintéticos
np.random.seed(42)
X = np.random.rand(1000, 2) * 10  # Mais pontos para melhor aprendizado
y = ((X[:, 0] + X[:, 1]) > 10).astype(int)

In [3]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Normalizar os dados
X_train, X_test = X_train / 10.0, X_test / 10.0

In [5]:
# Criar o modelo de rede neural com mais camadas e regularização
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(2,), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Definir e criar as pastas de saída se não existirem
frames_folder = os.path.join('..', 'frames')
media_folder = os.path.join('..', 'media')

if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)
    print(f"Pasta '{frames_folder}' criada com sucesso.")

if not os.path.exists(media_folder):
    os.makedirs(media_folder)
    print(f"Pasta '{media_folder}' criada com sucesso.")

In [7]:
# Treinar o modelo e capturar a evolução da fronteira de decisão
frames_list = [] # Renomeada para evitar conflito com a pasta 'frames'
for epoch in range(1, 21):  # Aumentar o número de épocas
    model.fit(X_train, y_train, epochs=1, verbose=0)
    
    # Gerar uma grade de pontos para visualizar a fronteira de decisão
    xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    preds = model.predict(grid).reshape(xx.shape)
    
    # Criar o frame da animação com tamanho ajustado para ser múltiplo de 16
    fig, ax = plt.subplots(figsize=(6.08, 6.08))  # 6.08 inches * 100 dpi = 608 pixels
    ax.contourf(xx, yy, preds, levels=[0, 0.5, 1], alpha=0.3, colors=['blue', 'red'])
    sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=y_train, palette=['blue', 'red'], edgecolor='k', s=50, legend=False)
    plt.title(f'Fronteira de Decisão - Época {epoch}')
    
    # Salvar frame na pasta 'frames'
    frame_path = os.path.join(frames_folder, f'frame_{epoch}.png')
    plt.savefig(frame_path)
    plt.close()
    
    frame = cv2.imread(frame_path)
    frames_list.append(frame) # Adicionar à lista de frames

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 636us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 615us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step


In [8]:
# Adicionar frame final com a mensagem de conclusão
fig, ax = plt.subplots(figsize=(6.08, 6.08))
ax.contourf(xx, yy, preds, levels=[0, 0.5, 1], alpha=0.3, colors=['blue', 'red'])
sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=y_train, palette=['blue', 'red'], edgecolor='k', s=50, legend=False)
plt.title('Separação das classes concluída!', fontsize=14)

# Salvar frame final na pasta 'frames'
final_frame_path = os.path.join(frames_folder, 'final_frame.png')
plt.savefig(final_frame_path)
plt.close()
final_frame = cv2.imread(final_frame_path)
frames_list.append(final_frame)

# Repetir a animação duas vezes (para um loop mais suave)
frames_list *= 7

In [9]:
# Criar animação GIF
gif_path = os.path.join(media_folder, 'evolucao_fronteira_decisao.gif')
imageio.mimsave(gif_path, frames_list, fps=5) # Ajuste o fps para controlar a velocidade

print(f'\nAnimação salva como {gif_path}')


Animação salva como ..\media\evolucao_fronteira_decisao.gif
